# Fake Covid News Detection

## Import Data

In [ ]:
import pandas as pd;
import re;
import os

if not os.path.exists('unclean'):
    os.makedirs('unclean')

test_df = pd.read_csv("unclean/Constraint_Test.csv");
train_df = pd.read_csv("unclean/Constraint_Train.csv");
val_df = pd.read_csv("unclean/Constraint_Val.csv");

In [110]:
test_df

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...
...,...,...
2135,2136,#CoronaVirusUpdates: State-wise details of Tot...
2136,2137,Tonight 12(midnight) onwards Disaster Manageme...
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 E...
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...


In [111]:
train_df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


In [112]:
val_df

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real
...,...,...,...
2135,2136,Donald Trump wrongly claimed that New Zealand ...,fake
2136,2137,Current understanding is #COVID19 spreads most...,real
2137,2138,Nothing screams “I am sat around doing fuck al...,fake
2138,2139,Birx says COVID-19 outbreak not under control ...,fake


# Explorative data analysis

In [113]:
print("Uncleaned Test Data: "+ str(test_df.shape))
print("Uncleaned Train Data: "+ str(train_df.shape))
print("Uncleaned Validate Data: " + str(val_df.shape))

Uncleaned Test Data: (2140, 2)
Uncleaned Train Data: (6420, 3)
Uncleaned Validate Data: (2140, 3)


In [114]:
print("Uncleaned Test Data: "+ str(test_df.columns))
print("Uncleaned Train Data: "+ str(train_df.columns))
print("Uncleaned Validate Data: " + str(val_df.columns))

Uncleaned Test Data: Index(['id', 'tweet'], dtype='object')
Uncleaned Train Data: Index(['id', 'tweet', 'label'], dtype='object')
Uncleaned Validate Data: Index(['id', 'tweet', 'label'], dtype='object')


In [115]:
test_df.info()
print('\n')
train_df.info()
print('\n')
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140 entries, 0 to 2139
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2140 non-null   int64 
 1   tweet   2140 non-null   object
dtypes: int64(1), object(1)
memory usage: 33.6+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6420 non-null   int64 
 1   tweet   6420 non-null   object
 2   label   6420 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.6+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140 entries, 0 to 2139
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2140 non-null   int64 
 1   tweet   2140 non-null   object
 2   label   2140 non-null   object
dtypes: int64(1), object(2)
memory usage: 50.3+ KB


# Data Preprocessing

## Convert to lowercase

In [116]:
train_df['tweet'] = train_df['tweet'].str.lower()
test_df['tweet'] = test_df['tweet'].str.lower()
val_df['tweet'] = val_df['tweet'].str.lower()
print(train_df.head())
print(test_df.head())
print(val_df.head())


   id                                              tweet label
0   1  the cdc currently reports 99031 deaths. in gen...  real
1   2  states reported 1121 deaths a small rise from ...  real
2   3  politically correct woman (almost) uses pandem...  fake
3   4  #indiafightscorona: we have 1524 #covid testin...  real
4   5  populous states can generate large case counts...  real
   id                                              tweet
0   1  our daily update is published. states reported...
1   2             alfalfa is the only cure for covid-19.
2   3  president trump asked what he would do if he w...
3   4  states reported 630 deaths. we are still seein...
4   5  this is the sixth time a global health emergen...
   id                                              tweet label
0   1  chinese converting to islam after realising th...  fake
1   2  11 out of 13 people (from the diamond princess...  fake
2   3  covid-19 is caused by a bacterium, not virus a...  fake
3   4  mike pence in rnc spe

## Check Duplicates

In [117]:
train_duplicates = train_df[train_df.duplicated(subset=['tweet'], keep=False)]
print("Duplicates in train_df:")
print(train_duplicates)


test_duplicates = test_df[test_df.duplicated(subset=['tweet'], keep=False)]
print("\nDuplicates in test_df:")
print(test_duplicates)


val_duplicates = val_df[val_df.duplicated(subset=['tweet'], keep=False)]
print("\nDuplicates in val_df:")
print(val_duplicates)

common_tweets_train_test = pd.merge(train_df, test_df, on='tweet', how='inner')
print("\nCommon tweets in train and test sets:")
print(common_tweets_train_test)

common_tweets_train_val = pd.merge(train_df, val_df, on='tweet', how='inner')
print("\nCommon tweets in train and validation sets:")
print(common_tweets_train_val)

common_tweets_test_val = pd.merge(test_df, val_df, on='tweet', how='inner')
print("\nCommon tweets in test and validation sets:")
common_tweets_test_val


Duplicates in train_df:
Empty DataFrame
Columns: [id, tweet, label]
Index: []

Duplicates in test_df:
Empty DataFrame
Columns: [id, tweet]
Index: []

Duplicates in val_df:
Empty DataFrame
Columns: [id, tweet, label]
Index: []

Common tweets in train and test sets:
   id_x                                              tweet label  id_y
0  3242  audio from whatsapp in which the head of cardi...  fake   132
1  4178  video shows muslim women spitting in plastic b...  fake  1829

Common tweets in train and validation sets:
   id_x                                              tweet label_x  id_y  \
0  5459  our combined total of confirmed and probable c...    real   910   

  label_y  
0    real  

Common tweets in test and validation sets:


,id_x,tweet,id_y,label


## Check Missing Values

In [118]:
print("Missing values in test_df:\n", test_df.isnull().sum())
print("\nMissing values in train_df:\n", train_df.isnull().sum())
print("\nMissing values in val_df:\n", val_df.isnull().sum())


Missing values in test_df:
 id       0
tweet    0
dtype: int64

Missing values in train_df:
 id       0
tweet    0
label    0
dtype: int64

Missing values in val_df:
 id       0
tweet    0
label    0
dtype: int64


## Categorical Values Count

In [119]:
val_df.label.value_counts()

,count
label,
real,1120
fake,1020


In [120]:
train_df.label.value_counts()

,count
label,
real,3360
fake,3060


## Remove extra whitespaces and line break

In [121]:
def count_extra_whitespace_and_linebreaks(df, column_name):
    extra_spaces_count = 0
    linebreaks_count = 0

    for index, row in df.iterrows():
        tweet = row[column_name]
        # Check for extra spaces (more than one space together)
        extra_spaces_count += len([i for i in tweet.split() if len(i) == 0])

        # Check for linebreaks
        linebreaks_count += tweet.count('\n')

    return extra_spaces_count, linebreaks_count

extra_spaces_test, linebreaks_test = count_extra_whitespace_and_linebreaks(test_df, 'tweet')
extra_spaces_train, linebreaks_train = count_extra_whitespace_and_linebreaks(train_df, 'tweet')
extra_spaces_val, linebreaks_val = count_extra_whitespace_and_linebreaks(val_df, 'tweet')

print("Test DataFrame:")
print(f"Extra spaces: {extra_spaces_test}")
print(f"Line breaks: {linebreaks_test}")

print("\nTrain DataFrame:")
print(f"Extra spaces: {extra_spaces_train}")
print(f"Line breaks: {linebreaks_train}")

print("\nValidation DataFrame:")
print(f"Extra spaces: {extra_spaces_val}")
print(f"Line breaks: {linebreaks_val}")


Test DataFrame:
Extra spaces: 0
Line breaks: 288

Train DataFrame:
Extra spaces: 0
Line breaks: 778

Validation DataFrame:
Extra spaces: 0
Line breaks: 239


In [122]:
import re

def clean_text(text):
  # Remove extra whitespaces
  text = re.sub(r'\s+', ' ', text)
  # Remove leading/trailing whitespaces
  text = text.strip()
  return text

train_df['tweet'] = train_df['tweet'].apply(clean_text)
test_df['tweet'] = test_df['tweet'].apply(clean_text)
val_df['tweet'] = val_df['tweet'].apply(clean_text)


In [133]:
# Recheck for extra whitespaces and line breaks after cleaning
extra_spaces_test, linebreaks_test = count_extra_whitespace_and_linebreaks(test_df, 'tweet')
extra_spaces_train, linebreaks_train = count_extra_whitespace_and_linebreaks(train_df, 'tweet')
extra_spaces_val, linebreaks_val = count_extra_whitespace_and_linebreaks(val_df, 'tweet')

print("\nAfter Cleaning:")
print("Test DataFrame:")
print(f"Extra spaces: {extra_spaces_test}")
print(f"Line breaks: {linebreaks_test}")

print("\nTrain DataFrame:")
print(f"Extra spaces: {extra_spaces_train}")
print(f"Line breaks: {linebreaks_train}")

print("\nValidation DataFrame:")
print(f"Extra spaces: {extra_spaces_val}")
print(f"Line breaks: {linebreaks_val}")



After Cleaning:
Test DataFrame:
Extra spaces: 0
Line breaks: 0

Train DataFrame:
Extra spaces: 0
Line breaks: 0

Validation DataFrame:
Extra spaces: 0
Line breaks: 0


## Normalize Multiple Punctuation into single punctuation

In [123]:
def count_multiple_punctuation(text):
    # Regular expression to find multiple punctuation marks together
    pattern = r'[^\w\s]{2,}'  # Matches 2 or more non-word, non-space characters
    matches = re.findall(pattern, text)
    return len(matches)

# Apply the function to the 'tweet' column of each DataFrame
train_df['multiple_punctuation_count'] = train_df['tweet'].apply(count_multiple_punctuation)
test_df['multiple_punctuation_count'] = test_df['tweet'].apply(count_multiple_punctuation)
val_df['multiple_punctuation_count'] = val_df['tweet'].apply(count_multiple_punctuation)

print(train_df.head())
print(test_df.head())
print(val_df.head())

# Example: Check the total count of multiple punctuations in train set
total_multiple_punctuation_train = train_df['multiple_punctuation_count'].sum()
print(f"\nTotal multiple punctuations in train_df: {total_multiple_punctuation_train}")


   id                                              tweet label  \
0   1  the cdc currently reports 99031 deaths. in gen...  real   
1   2  states reported 1121 deaths a small rise from ...  real   
2   3  politically correct woman (almost) uses pandem...  fake   
3   4  #indiafightscorona: we have 1524 #covid testin...  real   
4   5  populous states can generate large case counts...  real   

   multiple_punctuation_count  
0                           0  
1                           1  
2                           1  
3                           1  
4                           1  
   id                                              tweet  \
0   1  our daily update is published. states reported...   
1   2             alfalfa is the only cure for covid-19.   
2   3  president trump asked what he would do if he w...   
3   4  states reported 630 deaths. we are still seein...   
4   5  this is the sixth time a global health emergen...   

   multiple_punctuation_count  
0                 

In [124]:
def replace_excessive_punctuation(text):
    text = re.sub(r'([!?.]){2,}', r'\1', text)
    return text

train_df['tweet'] = train_df['tweet'].apply(replace_excessive_punctuation)
test_df['tweet'] = test_df['tweet'].apply(replace_excessive_punctuation)
val_df['tweet'] = val_df['tweet'].apply(replace_excessive_punctuation)


In [125]:
def check_multiple_punctuation(df):

    # Regular expression to find two or more punctuation marks together
    pattern = r'[!?.]{2,}'

    # Find tweets with multiple punctuation marks
    multiple_punctuation_tweets = df[df['tweet'].str.contains(pattern, regex=True)]

    if multiple_punctuation_tweets.empty:
        return None  # Return None if no such rows are found.
    else:
        return multiple_punctuation_tweets

multiple_punctuation_train = check_multiple_punctuation(train_df)

if multiple_punctuation_train is not None:
    print("Tweets in train_df with multiple punctuation marks:")
    print(multiple_punctuation_train)
else:
    print("No tweets in train_df with multiple punctuation marks.")

multiple_punctuation_test = check_multiple_punctuation(test_df)
if multiple_punctuation_test is not None:
    print("\nTweets in test_df with multiple punctuation marks:")
    print(multiple_punctuation_test)
else:
    print("\nNo tweets in test_df with multiple punctuation marks.")

multiple_punctuation_val = check_multiple_punctuation(val_df)
if multiple_punctuation_val is not None:
    print("\nTweets in val_df with multiple punctuation marks:")
    print(multiple_punctuation_val)
else:
    print("\nNo tweets in val_df with multiple punctuation marks.")


No tweets in train_df with multiple punctuation marks.

No tweets in test_df with multiple punctuation marks.

No tweets in val_df with multiple punctuation marks.


## Check Emoji Usage

In [134]:
def contains_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return bool(emoji_pattern.search(text))

train_df['contains_emoji'] = train_df['tweet'].apply(contains_emoji)
test_df['contains_emoji'] = test_df['tweet'].apply(contains_emoji)
val_df['contains_emoji'] = val_df['tweet'].apply(contains_emoji)

print(train_df[train_df['contains_emoji'] == True])
print(test_df[test_df['contains_emoji'] == True])
print(val_df[val_df['contains_emoji'] == True])


        id                                              tweet label  \
15      16  as of 18 august 2020 8am till now there have b...  real   
60      61  as of today there are a total of 1339 @icmrdel...  real   
71      72  according to the state health department #hary...  real   
86      87  there is no one in new zealand receiving hospi...  real   
106    107  as of today there are 23917 active #covid19 ca...  real   
...    ...                                                ...   ...   
5620  5621  as of 17 july 10pm there are 684 active #covid...  real   
5789  5790  as of today our #recoveryrate for #covid19 cas...  real   
6250  6251  yes the state governments may issue stricter g...  real   
6257  6258  as of today 87604 8592 out of the 101951 total...  real   
6360  6361  as of today #covid19 fatality rate is 253 the ...  real   

      multiple_punctuation_count  contains_emoji  
15                             1            True  
60                             2            T

## Classify URL Link

In [127]:
def replace_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub("<URL>", text)

train_df['tweet'] = train_df['tweet'].apply(replace_urls)
test_df['tweet'] = test_df['tweet'].apply(replace_urls)
val_df['tweet'] = val_df['tweet'].apply(replace_urls)


In [128]:
def count_url_occurrences(df):
    return df['tweet'].str.count('<URL>').sum()

train_url_count = count_url_occurrences(train_df)
test_url_count = count_url_occurrences(test_df)
val_url_count = count_url_occurrences(val_df)

print(f"Number of URLs in train_df: {train_url_count}")
print(f"Number of URLs in test_df: {test_url_count}")
print(f"Number of URLs in val_df: {val_url_count}")


Number of URLs in train_df: 4395
Number of URLs in test_df: 1496
Number of URLs in val_df: 1497


## Remove Special Character

In [129]:
def remove_special_chars(text):
    # Keep only letters, numbers, spaces, @ and #
    text = re.sub(r'[^\w\s@#]', '', text)
    return text

train_df['tweet'] = train_df['tweet'].apply(remove_special_chars)
test_df['tweet'] = test_df['tweet'].apply(remove_special_chars)
val_df['tweet'] = val_df['tweet'].apply(remove_special_chars)


# Export

In [ ]:
if not os.path.exists('clean'):
    os.makedirs('clean')

train_df.to_csv('clean/train_cleaned.csv', index=False)
test_df.to_csv('clean/test_cleaned.csv', index=False)
val_df.to_csv('clean/val_cleaned.csv', index=False)

train_df.to_excel('clean/train_cleaned.xlsx', index=False)
test_df.to_excel('clean/test_cleaned.xlsx', index=False)
val_df.to_excel('clean/val_cleaned.xlsx', index=False)
